 ## Solving Capacitated Lot Sizing Problem using PuLP

In [1]:
import random as r
import numpy as np
import time
import pandas as pd
from collections import defaultdict

In [2]:
data = pd.read_excel("Data.xlsx",header=0)
invt_data = pd.read_excel("Initial Inventory.xlsx",header=0)

In [3]:
data.drop(columns=['Index Machine','Demand Mo.1', 'Demand Tu.1',
       'Demand We.1', 'Demand Th.1', 'Demand Fr.1', 'Demand Sa.1',
       'Demand Su.1', 'Demand Mo.2', 'Demand Tu.2', 'Demand We.2',
       'Demand Th.2', 'Demand Fr.2', 'Demand Sa.2', 'Demand Su.2',
       'Demand Mo.3', 'Demand Tu.3', 'Demand We.3', 'Demand Th.3',
       'Demand Fr.3', 'Demand Sa.3', 'Demand Su.3', 'Demand Mo.4',
       'Demand Tu.4', 'Demand We.4', 'Demand Th.4', 'Demand Fr.4',
       'Demand Sa.4', 'Demand Su.4', 'Demand Mo.5', 'Demand Tu.5',
       'Demand We.5', 'Demand Th.5', 'Demand Fr.5', 'Demand Sa.5',
       'Demand Su.5', 'Demand Mo.6', 'Demand Tu.6', 'Demand We.6',
       'Demand Th.6', 'Demand Fr.6', 'Demand Sa.6', 'Demand Su.6',
       'Demand Mo.7', 'Demand Tu.7', 'Demand We.7', 'Demand Th.7',
       'Demand Fr.7', 'Demand Sa.7', 'Demand Su.7', 'Demand Mo.8',
       'Demand Tu.8', 'Demand We.8', 'Demand Th.8', 'Demand Fr.8',
       'Demand Sa.8', 'Demand Su.8', 'Demand Mo.9', 'Demand Tu.9',
       'Demand We.9', 'Demand Th.9', 'Demand Fr.9', 'Demand Sa.9',
       'Demand Su.9', 'Demand Mo.10', 'Demand Tu.10', 'Demand We.10',
       'Demand Th.10', 'Demand Fr.10', 'Demand Sa.10', 'Demand Su.10',
       'Demand Mo.11','Total Time'], inplace=True)
data.head()

,Index Part,Part,Machine 1 \n(preferred),Setup time M1\n[Min],Setup costs M1,Inventory costs \nct/day,Reject rate [%],Machine availability\n[%],Cycle time\n[s],Req. Worker,...,Size container,Buffer Capacity,Demand Mo,Demand Tu,Demand We,Demand Th,Demand Fr,Demand Sa,Demand Su,Demand Week
0,1,Stoßfänger vorne A1,1,15,93,7.8,0.050000,90,76,1,...,6,672,458,465,481,560,298,0,0,2262
1,2,Stoßfänger vorne A2,1,15,93,8.2,0.060000,90,77,1,...,6,510,36,84,64,60,16,0,0,260
2,3,Schwellerverkleidung A1,1,15,83,4.0,0.066667,90,75,1,...,24,912,251,251,251,251,251,126,0,1381
3,4,Schwellerverkleidung A2,1,15,83,4.0,0.040018,90,63,1,...,24,360,23,23,23,23,23,11,0,126
4,5,Stoßfänger vorne B,2,15,72,6.7,0.050764,90,76,1,...,6,462,142,136,116,131,76,0,0,601


In [4]:
invt_data.head()

,Part,Safety stock,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8,Week 9,Week 10,Week 11
0,Stoßfänger vorne A1,50,614,552,252,576,372,459,386,336,597,151,667
1,Stoßfänger vorne A2,50,160,402,360,372,195,218,328,250,422,329,231
2,Schwellerverkleidung A1,150,562,705,435,270,428,499,525,444,424,426,345
3,Schwellerverkleidung A2,72,195,87,237,131,204,135,252,92,157,177,177
4,Stoßfänger vorne B,72,198,352,229,317,170,250,276,138,176,321,384


In [5]:
initial_inventory = invt_data['Week 1']
safety_stock = invt_data['Safety stock']

In [6]:
no_of_items = data['Part'].count()
no_of_machines = 7
no_of_period = 6

In [7]:
demand = [data.iloc[i][j] for i in range(no_of_items) for j in range(13,19)]

In [8]:
buffer_cap = data['Buffer Capacity']
setup_cost = data['Setup costs M1']
inventory_cost = data['Inventory costs \nct/day']
setup_time = data['Setup time M1\n[Min]']*60  # convert to seconds
production_time = data['Cycle time\n[s]']
container_size = data['Size container']
machine_preference = data['Machine 1 \n(preferred) ']

In [9]:
item_list = [t for t in range(1,no_of_items+1)]
period_list = [t for t in range(1,no_of_period+1)]
machine_list = [t for t in range(1,no_of_machines+1)]

machine_item_pair = [(i,j) for i,j in zip(machine_preference,item_list)]
item_period_pair = [(j,t) for j in item_list for t in period_list]
machine_period_pair = [(i,t) for i in machine_list for t in period_list]

In [10]:
def set_zero():
    return 0

In [11]:
bjt = {(j,t): d for (j,t),d in zip(item_period_pair,demand)}               # Demand of item i at period t;
bjt = defaultdict(set_zero,bjt)
Bj  = {j:c for j,c in zip(item_list,buffer_cap)}                           # Buffer Capacity for part j
Bj = defaultdict(set_zero,Bj)
fij = {(i,j): s for (i,j),s in zip(machine_item_pair,setup_cost)}          # Setup cost of item i on machine j;
fij = defaultdict(set_zero,fij)
cj = {j: s for j,s in zip(item_list,inventory_cost)}                       # Unit inventory cost of item i per period;
cj = defaultdict(set_zero,cj)
rzij = {(i,j): s for (i,j),s in zip(machine_item_pair,setup_time)}         # Setup time of item i on machine j;
rzij = defaultdict(set_zero,rzij)
ZZij = {(i,j): s for (i,j),s in zip(machine_item_pair,production_time)}    # Unit production time of item j on machine i;
ZZij = defaultdict(set_zero,ZZij)
sfj = {j:s for j,s in zip(item_list,container_size)}                       # Size of Container of product j
sfj = defaultdict(set_zero,sfj)
ssj = {j:s for j,s in zip(item_list,safety_stock)}                         # safety stock for product j

In [12]:
import pulp as pl
from pulp import *

In [13]:
model = LpProblem("CLSP",LpMinimize)

In [14]:
fgh = [[i,j] for i,j in zip(machine_preference,item_list)]

In [15]:
vars_list = [ele+[period_list[i]] for ele in fgh for i in range(len(period_list))]
variables = [tuple(var) for var in vars_list]

In [16]:
# i -> machine , j -> item , t -> period
qijt = LpVariable.dicts("Lotsize",(variables),0,None,LpInteger)
qijt = defaultdict(set_zero,qijt)

ljt  = LpVariable.dicts("Inventory",(item_period_pair),0,None,LpInteger)
ljt = defaultdict(set_zero,ljt)

zijt = LpVariable.dicts("Production",(variables),0,1,LpInteger)
zijt = defaultdict(set_zero,zijt)

zstar = LpVariable.dicts("Setup",(variables),0,1,LpInteger)
zstar = defaultdict(set_zero,zstar)

rrt = LpVariable.dicts("RemainingSetTime",(machine_period_pair),0,None)
rrt = defaultdict(set_zero,rrt)

rt = LpVariable.dicts("SetupTime",(machine_period_pair),0,None)
rt = defaultdict(set_zero,rt)

tit = LpVariable.dicts("ProdTime",(machine_period_pair),0,None)
tit = defaultdict(set_zero,tit)

y1 = LpVariable.dicts("BinarySetup",(variables),0,1,LpInteger)
y1 = defaultdict(set_zero,y1)

y2 = LpVariable.dicts("BinarySetTime",(machine_period_pair),0,1,LpInteger)
y2 = defaultdict(set_zero,y2)

y3 = LpVariable.dicts("BinaryRemTime",(machine_period_pair),0,1,LpInteger)
y3 = defaultdict(set_zero,y3)

# intvar = LpVariable.dicts("rand_int",(item_period_pair),0,None,LpInteger)

In [17]:
# adding initial inventory values
for i in range(len(item_list)):
    ljt[i+1,0] = initial_inventory[i]

In [18]:
SetupCost = lpSum([fij[i,j]*zstar[i,j,t] for i,j,t in variables])
HoldingCost = lpSum([round(cj[j])*[ljt[j,t] + ljt[j,t-1]] for j,t in item_period_pair])
TotalCost = SetupCost + HoldingCost

In [19]:
#Objective function
model += TotalCost, "Sum_of_Setup_&_Holding_Costs"

In [20]:
# constraint 1
for j,t in item_period_pair:
    model += (ljt[j,t-1] + lpSum([qijt[i,j,t] for i in machine_list]) - ljt[j,t]) == bjt[j,t]

In [21]:
# constraint 2
for i,j,t in variables:
    model += (qijt[i,j,t])<=zijt[i,j,t]*lpSum([bjt[j,t]])

In [22]:
# constraint 3
L = max(demand)*5
for i,j,t in variables:
    model += (qijt[i,j,t] + zstar[i,j,t]) <= L*zijt[i,j,t]

In [23]:
# constraint 4
M = 50
for i,j,t in variables:
    model += (zstar[i,j,t]) >= zijt[i,j,t] - zijt[i,j,t-1]
    model += (zstar[i,j,t]) <= (zijt[i,j,t] - zijt[i,j,t-1]) + M*(1 - y1[i,j,t])
    model += (zstar[i,j,t]) <= M*y1[i,j,t]

In [24]:
# # constraint 5 (Doesn't make sense for single machine)
# for j,t in item_period_pair:
#     model += (lpSum([zijt[i,j,t] for i in machine_list])) <= 1    

In [25]:
# # constraint 6 (Doesn't make sense for single machine)
# for j,t in item_period_pair:
#     model += (lpSum([zstar[i,j,t] for i in machine_list])) <= 1

In [26]:
# constraint 9
N = no_of_period*90000
tstar = 86400
for i,t in machine_period_pair:
    model += (rt[i,t]) <= rrt[i,t] +  lpSum([zstar[i,j,t]*rzij[i,j] for j in item_list])
    model += (rt[i,t]) <= tstar
    model += (rt[i,t]) >= rrt[i,t] +  lpSum([zstar[i,j,t]*rzij[i,j] for j in item_list]) - N*(1-y2[i,t])
    model += (rt[i,t]) >= tstar - N*y2[i,t]

In [27]:
# constraint 10
O = no_of_period*90000
for i,t in machine_period_pair:
    model += (rrt[i,t]) >= lpSum([zstar[i,j,t]*rzij[i,j] for j in item_list]) + rrt[i,t-1] - tstar
    model += (rrt[i,t]) <= lpSum([zstar[i,j,t]*rzij[i,j] for j in item_list]) + rrt[i,t-1] - tstar + O*(1-y3[i,t])
    model += (rrt[i,t]) <= O*y3[i,t]

In [28]:
# constraint 11, 86400 secs in 24 hrs
for i,t in machine_period_pair:
    model += (tit[i,t]+ rt[i,t]) <= tstar

In [29]:
# constraint 12
for i,t in machine_period_pair:
    model += (tit[i,t]) == lpSum([ZZij[i,j]*qijt[i,j,t] for j in item_list])

In [30]:
# # constraint 13
# for i,j,t in variables:
#     model += (qijt[i,j,t])==intvar[j,t]*sfj[j]

In [31]:
# constraint 14
for j,t in item_period_pair:
    model += (ljt[j,t]) <= Bj[j]

In [32]:
# # new constraint
# for i in item_list:
#     model += (ljt[i,no_of_period]) >= ssj[i]

In [33]:
# model.writeLP("CLSP_LP_Relaxation.lp",)

In [34]:
solver = PULP_CBC_CMD(msg=True)
model.solve(solver)
print("Status: ",LpStatus[model.status])
print(value(model.objective))

Status:  Optimal
80704.0


In [35]:
import csv

In [36]:
import datetime 
  
def convert(n): 
    return str(datetime.timedelta(seconds = n))

In [37]:
LotSize = {}
for i,j,t in variables:
    LotSize[qijt[i,j,t]] = value(qijt[i,j,t])

In [38]:
StartSetup = {}
StartProd = {}
RemainingSetupTime = {}
for i,t in machine_period_pair:
    StartSetup[i,t] = convert(value(rt[i,t]))
    StartProd[i,t] = convert(value(tit[i,t]))
    RemainingSetupTime[i,t] = convert(value(rrt[i,t]))


In [39]:
solution = pd.DataFrame.from_dict(StartSetup, orient='index',columns=['Setup Time'])

solution["Production Time"] = StartProd.values()

solution.iloc[0:6]

,Setup Time,Production Time
"(1, 1)",0:15:00,0:00:00
"(1, 2)",0:00:00,6:31:24
"(1, 3)",0:30:00,14:38:49
"(1, 4)",0:00:00,18:20:05
"(1, 5)",0:00:00,11:51:45
"(1, 6)",0:00:00,2:37:30


In [40]:
dummy = pd.DataFrame(columns=[1,2,3,4], index = range(1,7))
for j in [1,2,3,4]:
    for t in period_list:
        dummy[j].iloc[t-1] = value(qijt[1,j,t])
dummy

C:\Program Files\Python37\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,1,2,3,4
1,0,0,0,0
2,309,0,0,0
3,481,24,191,0
4,560,60,251,0
5,298,16,251,0
6,0,0,126,0


In [41]:
dummy2 = pd.DataFrame(columns=[1,2,3,4], index = range(1,7))
for j in [1,2,3,4]:
    for t in period_list:
        dummy2[j].iloc[t-1] = value(ljt[j,t])
dummy2

,1,2,3,4
1,156,124,311,172
2,0,40,60,149
3,0,0,0,126
4,0,0,0,103
5,0,0,0,80
6,0,0,0,69


import csv
 

with open('InitialSolution.csv','w') as f:
    f_write = csv.writer(f,delimiter= ":",quoting=csv.QUOTE_NONNUMERIC,quotechar='"')
    for sol in model.variables():
        f_write.writerow([sol.name,sol.varValue])     

In [42]:
len(model.variables())

870

In [43]:
model

CLSP:
MINIMIZE
16*Inventory_(1,_1) + 16*Inventory_(1,_2) + 16*Inventory_(1,_3) + 16*Inventory_(1,_4) + 16*Inventory_(1,_5) + 8*Inventory_(1,_6) + 10*Inventory_(10,_1) + 10*Inventory_(10,_2) + 10*Inventory_(10,_3) + 10*Inventory_(10,_4) + 10*Inventory_(10,_5) + 5*Inventory_(10,_6) + 8*Inventory_(11,_1) + 8*Inventory_(11,_2) + 8*Inventory_(11,_3) + 8*Inventory_(11,_4) + 8*Inventory_(11,_5) + 4*Inventory_(11,_6) + 14*Inventory_(12,_1) + 14*Inventory_(12,_2) + 14*Inventory_(12,_3) + 14*Inventory_(12,_4) + 14*Inventory_(12,_5) + 7*Inventory_(12,_6) + 14*Inventory_(13,_1) + 14*Inventory_(13,_2) + 14*Inventory_(13,_3) + 14*Inventory_(13,_4) + 14*Inventory_(13,_5) + 7*Inventory_(13,_6) + 20*Inventory_(14,_1) + 20*Inventory_(14,_2) + 20*Inventory_(14,_3) + 20*Inventory_(14,_4) + 20*Inventory_(14,_5) + 10*Inventory_(14,_6) + 20*Inventory_(15,_1) + 20*Inventory_(15,_2) + 20*Inventory_(15,_3) + 20*Inventory_(15,_4) + 20*Inventory_(15,_5) + 10*Inventory_(15,_6) + 14*Inventory_(16,_1) + 14*Inventory